# Map Storms
This will map out storms using the Azure Map service in the backend. You will need a key to do this and save it in the `.env` with `MAP_KEY=xxxxxxxxxxxxx`

In [1]:
from dotenv import load_dotenv
import requests, json
import os
import folium

load_dotenv()

mapKey=os.getenv('MAP_KEY')

# Get the Storms


In [2]:
url=f"https://atlas.microsoft.com/weather/tropical/storms/active/json?api-version=1.1&subscription-key={mapKey}"
storms_resp = requests.get(url)
print(storms_resp.status_code)
storms=storms_resp.json()

storm_list = []

for storm in storms['results']:
    year=storm['year']
    govId=storm['govId']
    basinId=storm['basinId']
    name=storm['name']
    print(name)
    
    storm_resp = requests.get(f"https://atlas.microsoft.com/weather/tropical/storms/forecasts/json?api-version=1.1&subscription-key={mapKey}&year={year}&basinId={basinId}&govId={govId}&windowGeometry=true")
    # write to local
    with open(f'local/{name}.json', 'w') as f:
        json.dump(storm_resp.json(), f, indent=4)
    for result in storm_resp.json()['results']:
        date=result['dateTime']
        maxWindGust=result['maxWindGust']['value']
        
        if result['window']['geometry']['type'] == 'Polygon':
            storm_list.append({'name':name,'geometry':result['window']['geometry']['coordinates'][0],'date':date,'maxWindGust':maxWindGust})
        elif result['window']['geometry']['type'] == 'MultiPolygon':
            for poly in result['window']['geometry']['coordinates']:
                storm_list.append({'name':name,'geometry':poly[0],'date':date,'maxWindGust':maxWindGust})



200
Isaac
Joyce
Kirk
JEBI
KRATHON


# Generate a Map
This uses Folium and there are others out there

In [3]:
# Create a map centered at an approximate location
m = folium.Map(location=[20, 0], zoom_start=2)

# Iterate through the storm_list and add polygons to the map
for storm in storm_list:
    polygon = [(coord[1],coord[0]) for coord in storm['geometry']]
    # Map the polygon with lat,long
    folium.Polygon(locations=polygon, color='red', fill=True, fill_color='blue', fill_opacity=0.5).add_to(m)
    folium.Marker(location=polygon[0], popup=f"Storm: {storm['name']} on {storm['date']} with max wind gusts of {storm['maxWindGust']} mph").add_to(m)
# Save the map to an HTML file
m.save('local/storm_map.html')

# Display the map in a Jupyter Notebook
m
